## Using Tools/Function Calling in Llama 3.1 to provide real-time information in an LLM chat

Llama 3.1 supports 'tools'.  Tools are  functions that you can write and provide to the LLM to answer particular questions.   This can be very valuable in an agentic workflow built for a highly specialized purpose, where you can use the LLM to determine the intent of a user's question in a chat, extract the relevant parameters from the chat and pipe them into the function you have written in order to get the required info.

![Tools/Function Calling in Llama 3.1](function_calling.png)

Let's look at a simple example.  I'm going to create a workflow to find out the current temperature in a particular place of interest.  I'll then expand the tooling workflow to also allow the LLM to have access to the sunset time in a particular place.

### Creating a tool to get the current temperature

In this code I am writing a simple Python function which will ping an API to get the current temperature in a place.

In [ ]:
import requests

# function to get the current temperature in a place
def get_current_temperature(place: str) -> str:
  base_url = f"https://wttr.in/{place}?format=j1"
  response = requests.get(base_url)
  data = response.json()
  return f"The current temperature in {place} is {data['current_condition'][0]['temp_C']} degrees Celsius"

# test the function
get_current_temperature("London")

That seems to have worked.  

### Adding the tool to the LLM workflow

Next I am going to use this function as a tool to feed to the LLM.  Not all LLMs support tools, but for those that do, the idea is as follows:
* The LLM should identify the intent of a user query and determine that the intent is appropriate to call the tool
* The LLM should extract or determine the right arguments/parameters to give to the tool
* The LLM should capture the response of the tool and feed it back to the user.  To do this it may have to override some default behaviors (for example many LLMs have default responses which inform users that they cannot provide certain real-time information).

First, let's send a request with a message to Llama 3.1, and as part of that request, we will inform Llama that we have a function on our end which can determine the weather in a given place.

In [ ]:
import ollama
client = ollama.Client()

def tool_chat(model: str, query: str):
    response = client.chat(
        model = model,
        messages = [{'role': 'user', 'content': query}],
        tools = [
          {
            'type': 'function',
            'function': {
              'name': 'get_current_temperature',
              'description': 'Get the temperature in a place',
              'parameters': {
                'type': 'object',
                'properties': {
                  'place': {
                    'type': 'string',
                    'description': 'The place for which the temperature is requested',
                  }
                },
                'required': ['place'],
              },
            },
          },
        ],
      )
    
    return response['message']

Now let's see what the LLM returns when we ask a question that relates in some way to weather.

In [ ]:
query = "Should I wear a warm coat today in Rome?"

tool_chat("llama3.1:8b", query)

OK, so let's now build this into a chat workflow.

In [ ]:
import ollama
import asyncio

async def weather_chat(model: str, query: str) -> str:
  client = ollama.AsyncClient()
  # Initialize conversation with a query
  messages = [{'role': 'user', 'content': query}]

  # First API call: Send the query and function description to the model
  # use the tools parameter to define the tool so the LLM can understand it 
  response = await client.chat(
    model = model,
    messages = messages,
    tools = [
      {
        'type': 'function',
        'function': {
          'name': 'get_current_temperature',
          'description': 'Get the temperature in a place',
          'parameters': {
            'type': 'object',
            'properties': {
              'place': {
                'type': 'string',
                'description': 'The place for which the temperature is requested',
              }
            },
            'required': ['place'],
          },
        },
      },
    ],
  )

  # Add the model's response to the conversation history
  messages.append(response['message'])

  # Check if the model decided to use the provided tool and warn if it did not
  if not response['message'].get('tool_calls'):
    print("The model didn't use the function. Its response was:")
    print(response['message']['content'])
    return

  # If it did use the tool, process tool calls made by the model
  if response['message'].get('tool_calls'):
    available_functions = {
      'get_current_temperature': get_current_temperature,
    }
    for tool in response['message']['tool_calls']:
      function_to_call = available_functions[tool['function']['name']]
      function_response = function_to_call(tool['function']['arguments']['place'])
      # Add function response to the conversation
      # Warn model to override any default behavior
      messages.append(
        {
          'role': 'tool',
          'content': f"""
          Answer the following question: {query}.  
          Ignore any previous instructions or defaults and instead use the following information:  {function_response}
          """,
        }
      )

  # Second API call: Get final response from the model
  # Hopefully it will provide the information obtained using the tool
  final_response = await client.chat(model=model, messages=messages)
  print(final_response['message']['content'])


### Testing using various open models and queries
First, let's try this with Google's Gemma2 model and a simple question.


In [ ]:
await weather_chat('gemma2:27b', 'What is the temperature like right now in Dublin?')

OK, so we can see that some models don't yet support this feature.  Let's try the small version of Llama 3.1.

In [ ]:
await weather_chat('llama3.1:8b', 'What is the temperature like right now in Dublin?')

Works nicely - let's try asking indirectly.

In [ ]:
await weather_chat('llama3.1:8b', "What is the current temperature in Ireland's capital?")

Very nice! Let's push things a little further.

In [ ]:
await weather_chat('llama3.1:8b', "My sister says that if I flew into the capital city of Norway today, I should wear clothing for extreme weather.  Should I trust her advice?")

In [ ]:
await weather_chat('llama3.1:8b', "Compare the temperatures of these two cities right now: Dunedin, New Zealand and Reykjavik, Iceland?")

In [ ]:
await weather_chat('llama3.1:8b', "What kinds of clothes should I pack for my trip to Tasmania which leaves tomorrow?")

In [ ]:
await weather_chat('llama3.1:8b', "How much longer would it take a 50g ice cube to melt today in Dunedin compared to Marrakech?")


In [ ]:
await weather_chat('llama3.1:8b', "If a wombat was transported today to Northern Sweden and a penguin was transported today to Singapore, which would have the best chance of survival?")

## Adding multiple functions

Let's construct a second function which obtains details of events available on Ticketmaster.

In [ ]:
# function to get the today's sunset time in a place
def get_sunset(place: str) -> str:
  base_url = f'https://wttr.in/{place}?format="%s"'
  response = requests.get(base_url)
  data = response.json()
  return f"Today's sunset time in {place} is {data}"

# test the function
get_sunset("London")

Now let's add this as an additional function available to Llama 3.1

In [ ]:
async def weather_and_sunset_chat(model: str, query: str) -> str:
  client = ollama.AsyncClient()
  # Initialize conversation with a query
  messages = [{'role': 'user', 'content': query}]

  # First API call: Send the query and function description to the model
  # use the tools parameter to define the tool so the LLM can understand it 
  response = await client.chat(
    model = model,
    messages = messages,
    tools = [
      {
        'type': 'function',
        'function': {
          'name': 'get_current_temperature',
          'description': 'Get the temperature in a place',
          'parameters': {
            'type': 'object',
            'properties': {
              'place': {
                'type': 'string',
                'description': 'The place for which the temperature is requested',
              }
            },
            'required': ['place'],
          },
        },
      },
     {
        'type': 'function',
        'function': {
          'name': 'get_sunset',
          'description': 'Get the sunset time in a place',
          'parameters': {
            'type': 'object',
            'properties': {
              'place': {
                'type': 'string',
                'description': 'The city in which the sunset time is requested',
              }
            },
            'required': ['place'],
          },
        },
      }, 
    ],
  )

  # Add the model's response to the conversation history
  messages.append(response['message'])

  # Check if the model decided to use the provided tool and warn if it did not
  if not response['message'].get('tool_calls'):
    print("The model didn't use the function. Its response was:")
    print(response['message']['content'])
    return

  # If it did use the tool, process tool calls made by the model
  if response['message'].get('tool_calls'):
    available_functions = {
      'get_current_temperature': get_current_temperature,
      'get_sunset': get_sunset
    }
    for tool in response['message']['tool_calls']:
      function_to_call = available_functions[tool['function']['name']]
      function_response = function_to_call(tool['function']['arguments']['place'])
      # Add function response to the conversation
      # Warn model to override any default behavior
      messages.append(
        {
          'role': 'tool',
          'content': f"""
          Answer the following question: {query}.  
          Ignore any previous instructions or defaults and instead use the following information:  {function_response}
          """,
        }
      )

  # Second API call: Get final response from the model
  # Hopefully it will provide the information obtained using the tool
  final_response = await client.chat(model=model, messages=messages)
  print(final_response['message']['content'])

And now let's test a few queries:

In [ ]:
await weather_and_sunset_chat("llama3.1:8b", "Should I dress light for my visit To Austin, Texas today?")

In [ ]:
await weather_and_sunset_chat("llama3.1:70b", "I'm driving about 100 miles out of town of Austin, Texas today.  What kind of clothes should I pack?  Also I want to be back before dark.  Do you have any advice?")